# Bağlantı

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls

adc.json  drive  sample_data


# Kütüphane

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import string #noktalama işaretleri
import re
import nltk #etkisiz kelimeler
import numpy
import csv
import pandas as pd
import os, glob
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import requests




# Test için örnek dava

In [ ]:

# excel'den veri yükleme

from openpyxl import Workbook,load_workbook 
hukuk = pd.read_excel('/content/drive/Colab Notebooks/Hukuk/hukukk.xlsx')

#ilk beş girdi görmek için
hukuk.head() 

,Suc,Karar,Hukum
0,"Resmi belgede sahtecilik, Bedelsiz kalmış sene...","Sanığın, borcuna karşılık olarak katılana verd...",Beraat
1,Kasten yaralama,Sanık hakkında tekerrüre esas alınan ilamdaki ...,Mahkumiyet
2,Resmi belgede sahtecilik,1-Suça konu bonoların katılan tarafından iki i...,Mahkumiyet
3,"Kasten yaralama, 6136 sayılı kanuna aykırılık",Mahalli mahkemece verilen hükümler temyiz edil...,Mahkumiyet
4,TAPU İPTALİ VE TESCİL-TENKİS,"Taraflar arasında görülen davada;\nDavacı, mir...","Hükmüne uyulan bozma kararında, gösterildiği ş..."


# Veri Seti Önişleme

In [ ]:

nltk.download('stopwords') # önemliiiii
noktalama = string.punctuation
etkisiz = stopwords.words('turkish')
print(noktalama)
print(etkisiz)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']


# Eğitim Verileri

In [ ]:
dizi= []
for d in hukuk['Karar'].head():

    print(d+ '\n------------------------------')
    # Etkisiz kelimeleri atmak
    temp= ' '
    for word in d.split():
        if word not in etkisiz and not word.isnumeric():
            temp += word + ' '
      
    print(temp+ '\n********************')
    dizi.append(temp)
   

Sanığın, borcuna karşılık olarak katılana verdiği 15.07.2009 tarihli 9.543 TL bedelli çeki ödeyip katılandan geri aldığı, katılanın çekin arkasında cirosunun bulunduğu, sanığın geri aldığı çekin miktar kısmını 39.543 TL olarak, tarihini de 15.09.2009 olarak değiştirdikten sonra yeninden ciro ederek ...’a verdiği, çekin ... tarafından katılan ve sanık aleyhine icraya konulduğu, sanığın çek üzerinde tahrifat yapmak suretiyle resmi belgede sahtecilik suçunu işlediği iddiasıyla açılan kamu davasında; katılanın sanıktan aldığı çeki ciro ederek borcuna karşılık ... isimli kişiye verdiği daha sonra borcunu ödeyerek geri aldığı çeki, arkasında kendi cirosu bulunduğu halde sanığa geri verdiği ve karşılığında demir alarak çek bedelinin katılana ödendiği, katılanın bu çek sebebiyle herhangi bir borcunun olmadığının sanık tarafından bilindiği halde, çek üzerinde katılanın bilgisi ve rızası olmaksızın miktar ve tarih kısmının değiştirilerek başkasına ciro edilerek katılanın borçlu konumuna getirild

In [ ]:
for d in hukuk['Karar'].head():
    print(d+ '\n------------------------------')
    # Noktalama işaretleri atma
    temp= ''
    for word in d:
        if word not in noktalama:
            temp += word
    print(temp+ '\n********************')
    

Sanığın, borcuna karşılık olarak katılana verdiği 15.07.2009 tarihli 9.543 TL bedelli çeki ödeyip katılandan geri aldığı, katılanın çekin arkasında cirosunun bulunduğu, sanığın geri aldığı çekin miktar kısmını 39.543 TL olarak, tarihini de 15.09.2009 olarak değiştirdikten sonra yeninden ciro ederek ...’a verdiği, çekin ... tarafından katılan ve sanık aleyhine icraya konulduğu, sanığın çek üzerinde tahrifat yapmak suretiyle resmi belgede sahtecilik suçunu işlediği iddiasıyla açılan kamu davasında; katılanın sanıktan aldığı çeki ciro ederek borcuna karşılık ... isimli kişiye verdiği daha sonra borcunu ödeyerek geri aldığı çeki, arkasında kendi cirosu bulunduğu halde sanığa geri verdiği ve karşılığında demir alarak çek bedelinin katılana ödendiği, katılanın bu çek sebebiyle herhangi bir borcunun olmadığının sanık tarafından bilindiği halde, çek üzerinde katılanın bilgisi ve rızası olmaksızın miktar ve tarih kısmının değiştirilerek başkasına ciro edilerek katılanın borçlu konumuna getirild

# Verileri Kayıt etme

In [ ]:
hukuk.to_csv('/content/drive/Colab Notebooks/Hukuk/cleaned.csv', index = False)

In [ ]:
hukuk = pd.read_csv('/content/drive/Colab Notebooks/Hukuk/cleaned.csv')
print(hukuk.head())

                                                 Suc  ...                                              Hukum
0  Resmi belgede sahtecilik, Bedelsiz kalmış sene...  ...                                             Beraat
1                                    Kasten yaralama  ...                                         Mahkumiyet
2                           Resmi belgede sahtecilik  ...                                        Mahkumiyet 
3      Kasten yaralama, 6136 sayılı kanuna aykırılık  ...                                         Mahkumiyet
4                       TAPU İPTALİ VE TESCİL-TENKİS  ...  Hükmüne uyulan bozma kararında, gösterildiği ş...

[5 rows x 3 columns]


# Veri Setini Bölme

In [ ]:
# Arındırılmış veriyi train ve test kümelerine ayırıyoruz
# Eğitim ve test olarak ayıracağız. train=eğitim, test=test

X_train, X_test, y_train, y_test = train_test_split(hukuk['Karar'].values.astype('U'),hukuk['Hukum'].values.astype('U'), test_size=0.20, random_state=1)

print(X_train.shape)
print(X_test.shape)                                                                      

(43,)
(11,)


# Sayma Vektörü Oluşturma

In [ ]:
# Train kümesindeki cümlelerin sayma vektöelerini çıkarıyoruz
count_vect = CountVectorizer(max_features = 2000000) # Burası önemli ram için
X_train_counts = count_vect.fit_transform(X_train)
print(X_train_counts.shape)

(43, 3031)


# Tf*Idf vektörü oluşturma sadece o "karara" özgü(temsil edecek kelimler olmalı)

In [ ]:
# Train kümesindeki cümlelerin TF*IDF vektörlerini sayma vekttörlerinden oluşturuyoruz
# Tf= terim sıklığı, IDF= bir kelimenin dökümanda kaç kere geçtiği
# TF*IDF bütün olarak; bir kelimenin bir döküman içinde ki önemini gösterir
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

(43, 3031)


# Naive Bayes Model Eğitimi

In [ ]:
# Çok modlu Naive Bayes Sınıflandırıcısı eğitiyoruz
clf = MultinomialNB().fit(X_train_tfidf, y_train)
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

# Model Performansı Ölçme

In [ ]:
# Sınıflandırıcı ile test seti üzerindeki tahminleme yapıyoruz
y_pred = clf.predict(X_test_tfidf)
for review, sentiment in zip(X_test[:], y_pred[:]):
  print('%r => %s' % (review, sentiment))

"6545 sayılı Kanun'un 81. maddesi ile değişik 5275 sayılı Kanun'un 106/3. maddesinde yer alan “Hükümlü, tebliğ olunan ödeme emri üzerine belli süre içinde adli para cezasını ödemezse, Cumhuriyet savcısının kararı ile ödenmeyen kısma karşılık gelen gün miktarı hapis cezasına çevrilerek...” ve 5237 sayılı TCK'nın 52/4. maddesindeki ''... kararda taksitlerden birinin zamanında ödenmemesi halinde geri kalan kısmının tamamının tahsil edileceği ve ödenmeyen adli para cezasının hapse çevrileceği belirtilir.'' şeklindeki düzenlemeler gözetildiğinde, tebliğnamedeki “ödenmeyen adli para cezasının hapse çevrileceğinin ihtarına” kısmının hüküm fıkrasından çıkartılmasına ilişkin görüşe iştirak edilmemiştir.\nTCK'nın 53. maddesinin uygulanması ile ilgili olarak Anayasa Mahkemesi'nin 08.10.2015 tarih, 2014/140 esas ve 2015/85 sayılı kararı ve 7242 sayılı Kanun'la yapılan değişikliklerin infaz aşamasında gözetilmesi mümkün görülmüştür.\nYargılama sürecindeki işlemlerin kanuna uygun olarak yapıldığı, d

In [ ]:
# Performans sonuçları
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))

0.2727272727272727
